## Example2

In [40]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

spark = SparkSession.builder.appName("SparkSql").getOrCreate()


people = spark.read.option("header","true").option("inferschema","true").\
            csv("../DATA/fakefriends_header.csv")
schemapeople.createOrReplaceTempView("people")


In [20]:
people.printSchema()

root
 |-- userID: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- friends: integer (nullable = true)



In [22]:
people.select("name").show()

+--------+
|    name|
+--------+
|    Will|
|Jean-Luc|
|    Hugh|
|  Deanna|
|   Quark|
|  Weyoun|
|  Gowron|
|    Will|
|  Jadzia|
|    Hugh|
|     Odo|
|     Ben|
|   Keiko|
|Jean-Luc|
|    Hugh|
|     Rom|
|  Weyoun|
|     Odo|
|Jean-Luc|
|  Geordi|
+--------+
only showing top 20 rows



In [23]:
people.filter(people.age>20).show()

+------+--------+---+-------+
|userID|    name|age|friends|
+------+--------+---+-------+
|     0|    Will| 33|    385|
|     1|Jean-Luc| 26|      2|
|     2|    Hugh| 55|    221|
|     3|  Deanna| 40|    465|
|     4|   Quark| 68|     21|
|     5|  Weyoun| 59|    318|
|     6|  Gowron| 37|    220|
|     7|    Will| 54|    307|
|     8|  Jadzia| 38|    380|
|     9|    Hugh| 27|    181|
|    10|     Odo| 53|    191|
|    11|     Ben| 57|    372|
|    12|   Keiko| 54|    253|
|    13|Jean-Luc| 56|    444|
|    14|    Hugh| 43|     49|
|    15|     Rom| 36|     49|
|    16|  Weyoun| 22|    323|
|    17|     Odo| 35|     13|
|    18|Jean-Luc| 45|    455|
|    19|  Geordi| 60|    246|
+------+--------+---+-------+
only showing top 20 rows



In [25]:
people.groupBy("age").count().show()

+---+-----+
|age|count|
+---+-----+
| 31|    8|
| 65|    5|
| 53|    7|
| 34|    6|
| 28|   10|
| 26|   17|
| 27|    8|
| 44|   12|
| 22|    7|
| 47|    9|
| 52|   11|
| 40|   17|
| 20|    5|
| 57|   12|
| 54|   13|
| 48|   10|
| 19|   11|
| 64|   12|
| 41|    9|
| 43|    7|
+---+-----+
only showing top 20 rows



# excersise 1

In [41]:
from pyspark.sql import functions as func

In [42]:
people.groupBy("age").agg({"friends":"mean"}).sort("age").show()

+---+------------------+
|age|      avg(friends)|
+---+------------------+
| 18|           343.375|
| 19|213.27272727272728|
| 20|             165.0|
| 21|           350.875|
| 22|206.42857142857142|
| 23|             246.3|
| 24|             233.8|
| 25|197.45454545454547|
| 26|242.05882352941177|
| 27|           228.125|
| 28|             209.1|
| 29|215.91666666666666|
| 30| 235.8181818181818|
| 31|            267.25|
| 32| 207.9090909090909|
| 33| 325.3333333333333|
| 34|             245.5|
| 35|           211.625|
| 36|             246.6|
| 37|249.33333333333334|
+---+------------------+
only showing top 20 rows



In [57]:
people.groupBy("age").\
    agg(func.round(
                    func.avg("friends"), 2).\
                    alias("friends_avg")).\
    sort(func.desc("friends_avg")).show()

+---+-----------+
|age|friends_avg|
+---+-----------+
| 63|      384.0|
| 21|     350.88|
| 18|     343.38|
| 52|     340.64|
| 33|     325.33|
| 45|     309.54|
| 56|     306.67|
| 42|      303.5|
| 51|     302.14|
| 65|      298.2|
| 55|     295.54|
| 44|     282.17|
| 48|      281.4|
| 64|     281.33|
| 54|     278.08|
| 66|     276.44|
| 68|      269.6|
| 41|     268.56|
| 31|     267.25|
| 57|     258.83|
+---+-----------+
only showing top 20 rows



In [58]:
spark.stop()

# example 3

In [62]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

spark = SparkSession.builder.appName("wordCount").getOrCreate()

In [99]:
book = spark.read.text("/media/user/Files/sparkCourse/DATA/book")

In [100]:
words = book.select(func.explode(func.split(book.value,r'\W+')).alias("word"))

In [101]:
words =words.filter(words.word!="")
words =words.select(func.lower(words.word).alias("word"))

In [102]:
words.groupBy("word").agg(func.count("word").alias("wordCount")).sort(func.desc("wordCount")).show()

+--------+---------+
|    word|wordCount|
+--------+---------+
|     you|     1878|
|      to|     1828|
|    your|     1420|
|     the|     1292|
|       a|     1191|
|      of|      970|
|     and|      934|
|    that|      747|
|      it|      649|
|      in|      616|
|      is|      560|
|     for|      537|
|      on|      428|
|     are|      424|
|      if|      411|
|       s|      391|
|       i|      387|
|business|      383|
|     can|      376|
|      be|      369|
+--------+---------+
only showing top 20 rows



In [103]:
spark.stop()

# example 4

In [118]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructField, StructType,StringType,IntegerType,FloatType

In [119]:
spark = SparkSession.builder.appName("tempApp").getOrCreate()
schema = StructType([
    StructField("ID",StringType(),True),
    StructField("date",IntegerType(),True),
    StructField("measureType",StringType(),True),
    StructField("temp",FloatType(),True)
])

In [120]:
data = spark.read.schema(schema).csv("../DATA/1800.csv")

In [121]:
data.show()

+-----------+--------+-----------+------+
|         ID|    date|measureType|  temp|
+-----------+--------+-----------+------+
|ITE00100554|18000101|       TMAX| -75.0|
|ITE00100554|18000101|       TMIN|-148.0|
|GM000010962|18000101|       PRCP|   0.0|
|EZE00100082|18000101|       TMAX| -86.0|
|EZE00100082|18000101|       TMIN|-135.0|
|ITE00100554|18000102|       TMAX| -60.0|
|ITE00100554|18000102|       TMIN|-125.0|
|GM000010962|18000102|       PRCP|   0.0|
|EZE00100082|18000102|       TMAX| -44.0|
|EZE00100082|18000102|       TMIN|-130.0|
|ITE00100554|18000103|       TMAX| -23.0|
|ITE00100554|18000103|       TMIN| -46.0|
|GM000010962|18000103|       PRCP|   4.0|
|EZE00100082|18000103|       TMAX| -10.0|
|EZE00100082|18000103|       TMIN| -73.0|
|ITE00100554|18000104|       TMAX|   0.0|
|ITE00100554|18000104|       TMIN| -13.0|
|GM000010962|18000104|       PRCP|   0.0|
|EZE00100082|18000104|       TMAX| -55.0|
|EZE00100082|18000104|       TMIN| -74.0|
+-----------+--------+-----------+

In [122]:
data =data.filter(data.measureType=="TMIN")

In [124]:
data.groupBy("ID").agg(func.min("temp").alias("min_temp")).sort("min_temp").show(20)

+-----------+--------+
|         ID|min_temp|
+-----------+--------+
|ITE00100554|  -148.0|
|EZE00100082|  -135.0|
+-----------+--------+



In [125]:
spark.stop()

# excersise 2

In [126]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as func
from pyspark.sql.types import StructField, StructType,IntegerType,FloatType

In [130]:
spark = SparkSession.builder.appName("sales").getOrCreate()
schema = StructType([
    StructField("customer_id", IntegerType(),True),
    StructField("item_id",  IntegerType(),True),
    StructField("price",FloatType (),True)
])


In [131]:
data = spark.read.schema(schema).csv("../DATA/customer-orders.csv")

In [132]:
data.show()

+-----------+-------+-----+
|customer_id|item_id|price|
+-----------+-------+-----+
|         44|   8602|37.19|
|         35|   5368|65.89|
|          2|   3391|40.64|
|         47|   6694|14.98|
|         29|    680|13.08|
|         91|   8900|24.59|
|         70|   3959|68.68|
|         85|   1733|28.53|
|         53|   9900|83.55|
|         14|   1505| 4.32|
|         51|   3378| 19.8|
|         42|   6926|57.77|
|          2|   4424|55.77|
|         79|   9291|33.17|
|         50|   3901|23.57|
|         20|   6633| 6.49|
|         15|   6148|65.53|
|         44|   8331|99.19|
|          5|   3505|64.18|
|         48|   5539|32.42|
+-----------+-------+-----+
only showing top 20 rows



In [136]:
data.groupBy("customer_id").agg(func.round(func.sum("price"),2).alias("total_price")).sort(func.desc("total_price")).show()

+-----------+-----------+
|customer_id|total_price|
+-----------+-----------+
|         68|    6375.45|
|         73|     6206.2|
|         39|    6193.11|
|         54|    6065.39|
|         71|    5995.66|
|          2|    5994.59|
|         97|    5977.19|
|         46|    5963.11|
|         42|    5696.84|
|         59|    5642.89|
|         41|    5637.62|
|          0|    5524.95|
|          8|    5517.24|
|         85|    5503.43|
|         61|    5497.48|
|         32|    5496.05|
|         58|    5437.73|
|         63|    5415.15|
|         15|    5413.51|
|          6|    5397.88|
+-----------+-----------+
only showing top 20 rows



In [137]:
spark.stop()